## The project: Bilateral filtering on resliced images 

This project aims to implement and then use a bilateral filtering method, described in the paper “Bilateral
Filtering for Gray and Color Images”, applied on either resampled 2D images or the entire 3D image. The
submitted report should contain, but not limited to, the following aspects

### Implementation of a re-slicing algorithm that obtains an image slice in a non-orthogonal plane, reasonable for viewing prostate gland in 2D images